# signac Projectile Demo (Brief)

*The following cell resets all data from previous runs of this notebook.*

In [ ]:
!rm -rf workspace signac.rc project.py dashboard.py *.err.* *.out.* signac_project_document.json view

## Introduction

This notebook gives an example of how the ``signac`` framework can be used to manage a data space and automate operations on this data space.

In this example, let's imagine that we're studying the behavior of a projectile launched at a specific velocity and angle to visualize the distance it will travel before it lands.
We use simple Newtonian mechanics to model the motion to determine how long the object travels: 

$$y(t) = y(0) + v\sin(\theta) t - \frac{1}{2} g t^2$$

Setting $y(0)=0$ and solving for $t_{max}$ such that $y(t_{max}) = 0$ yields: $t_\max= \frac{2v \sin(\theta)}{g}$

## Initial experiments

We express the simple math from above in two Python functions that calculate the maximum time the projectile travels, $t_\max$, and the $(x, y)$ coordinates of its trajectory.

In [ ]:
import numpy as np

def get_t_max(v, theta, g=9.81):
    return 2 * v * np.sin(theta) / g

def compute_xy(t, v, theta, g=9.81):
    return v * np.cos(theta) * t, v * np.sin(theta) * t - (g/2) * t**2

Let's observe the effect of launching the projectile at different angles:

In [ ]:
theta = 20 * np.pi / 180   # rad

t_max = get_t_max(
    v     = 2000,  # m/s
    theta = theta,
)

print("Time traveled (theta={:2.1f}): {:.2f} min".format(theta * 180/np.pi, t_max / 60))

We can also execute a slightly more "*systematic*" study of the maximum distance traveled over different launch angles:

In [ ]:
for theta in np.arange(0.0, np.pi/2, 0.2):
    t_max = get_t_max(2000, theta)
    x_max = compute_xy(t_max, 2000, theta)[0]
    print("Distance traveled (theta={:04.1f}): {:3.2f} km".format(theta * 180/np.pi, x_max / 1000))

We have a plotting function in the `render.py` module:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from render import plot

fig, ax = plot(velocity=2000, theta=45 * np.pi / 180)
plt.show()

## Initialize a data space

So far so good, but now, let's see how we can manage this data with ``signac``. 

In [ ]:
import signac

# Start by initializing a project
project = signac.init_project("Projectile-Project")

# Obtain a 'job' handle for a specific *state point*:
job = project.open_job({"theta": 0.4, "velocity": 2000})

In [ ]:
# JSON-encodable data can be stored in the *job document*, which works like a persistent dict:
job.doc['t_max'] = get_t_max(v=job.sp['velocity'], theta=job.sp['theta'])

# Just like the state point, the document data can also be accessed via *attributes*:
job.doc.x_max = compute_xy(t=job.doc.t_max, v=job.sp.velocity, theta=job.sp.theta)[0]

In [ ]:
print(job.sp)
print(job.doc)

A *job* essentially represents a directory within our *workspace* on the file system.

In [ ]:
print(job.get_id())
print(job.workspace())

We can use that to manage files directory on the file system:

In [ ]:
fig, ax = plot(velocity=job.sp.velocity, theta=job.sp.theta)

fig.savefig(job.fn('trajectory.png'))

The `job.fn('trajectory.png')` command is a short-cut for `os.path.join(job.workspace(), 'trajectory.png')`.

We have now created the following directory structure on the file system:

In [ ]:
! find . -not -path '*/[\._]*'

## Expand data space

We've shown how this works for one data point.

However, `signac` is designed to interact with large data spaces with lots of data points.
This is useful, for example to conduct a parameter study of various launch angles:

In [ ]:
for velocity in 2000, 2500, 3000:
    for theta in 0.4, 0.625, 0.85, 1.3:
        job = project.open_job({"velocity": velocity, "theta": theta})
        job.doc.t_max = get_t_max(v=job.sp.velocity, theta=job.sp.theta)
        job.doc.x_max = compute_xy(t=job.doc.t_max, v=job.sp.velocity, theta=job.sp.theta)[0]

## Accessing this data

The data is stored persistently on the file system and can be accessed later, for example, by iterating over the entire project.

In [ ]:
x_max = 0
theta_max = 0

for job in project:
    if job.doc.x_max > x_max:
        x_max = job.doc.x_max
        theta_max = job.sp.theta

print("The furthest distance traveled was {:3.2f} km with \u03b8={:04.1f}\u00b0.".format(
    x_max/1000, theta_max*180/np.pi))

## Defining workflows with signac-flow

Since we're now working with a larger data space, it is a good idea to automate our workflow.
For this we define a `FlowProject` and functions that *operate* on the data space as part of a workflow.
We define `MyProject` as a class that inherits its behavior directly from the `FlowProject`.

In [ ]:
%%writefile project.py
from flow import FlowProject
import numpy as np


def get_t_max(v, theta, g=9.81):
    return 2 * v * np.sin(theta) / g


def compute_xy(t, v, theta, g=9.81):
    return v * np.cos(theta) * t, v * np.sin(theta) * t - (g/2) * t**2


from flow import FlowProject

class MyProject(FlowProject):
    pass


@MyProject.label
def trajectory_computed(job):
    return job.isfile('trajectory.npz')


@MyProject.operation
@MyProject.post(trajectory_computed)
def compute_trajectory(job):
    from time import sleep; sleep(1)  # add some artificial computational cost here
    t = np.linspace(0, get_t_max(job.sp.velocity, job.sp.theta), 100)
    xy = np.asarray(compute_xy(t, job.sp.velocity, job.sp.theta)).T
    np.savez(job.fn('trajectory.npz'), t=t, xy=xy)
    
    job.doc.t_max = t.max()
    job.doc.x_max = xy[:, 0].max()


@MyProject.operation
@MyProject.pre.after(compute_trajectory)
@MyProject.post.isfile('trajectory.png')
def plot_trajectory(job):
    from render import plot
    fig, ax = plot(velocity=job.sp.velocity, theta=job.sp.theta)
    fig.savefig(job.fn('trajectory.png'))


if __name__ == '__main__':
    MyProject().main()

In [ ]:
!python3 project.py status --full --pretty --parameters velocity theta --stack

In [ ]:
!python3 project.py run -o compute_trajectory --progress

In [ ]:
!python3 project.py run --progress --parallel=4

In [ ]:
!python3 project.py status -d --only-incomplete

## Data visualization with signac-dashboard

Below, we define and run an instance of ``signac-dashboard`` to visualize the data.

In [ ]:
%%writefile dashboard.py
from signac_dashboard import Dashboard
from signac_dashboard.modules import StatepointList, DocumentList, ImageViewer

class ProjectileDashboard(Dashboard):
    pass

modules = [StatepointList(),
           DocumentList(),
           ImageViewer()]

if __name__ == '__main__':
    ProjectileDashboard(modules=modules).main()

In [ ]:
!python3 dashboard.py run